In [1]:
import matplotlib.pyplot as plt
import numpy as np
import json
import os
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [2]:
funcs = [
    lambda x, y: np.exp(y+np.sin(x)),
    lambda x, y: (x-0.5)**2+(y-0.5)**2,
    lambda x, y: np.sin(2*np.pi*x)*np.sin(2*np.pi*y)
]
nRange = [8, 16, 32, 64]
boundaryTypes = [
    ["Dirichlet", "Neumann"],
    ["Dirichlet", "DirichletNeumann", "NeumannDirichlet", "Neumann"]
]

Call Microsolvt

In [ ]:
for funcId in range(3):
    for regionId in range(2):
        for boundaryType in boundaryTypes[regionId]:
            for n in nRange:
                prefix = f'Func{funcId}_{"HasCircle" if regionId else "NoCircle"}_{boundaryType}_Grid{n}'
                with open("input.json","w") as f:
                    params = {
                        "grid_h": 1.0 / n,
                        "bdry_type": boundaryType,  
                        "func_idx": funcId,
                        "output_path": f'result/{prefix}.txt'
                    }
                    if regionId:
                        params["circ_c"] = [0.5, 0.5]
                        params["circ_r"] = 0.35
                    f.write(json.dumps(params, indent=4, separators=(',', ': ')))
                os.system('./build/Microsolvt')

Draw images and calculate norms

In [3]:
def analyze(funcId, stride, inputPath, outputPath):
    with open(inputPath) as f:
        # our solution
        vals = np.array([[float(s) if s != '*' else np.inf for s in lines[:-2].split(' ')] for lines in f.readlines()])
        exterior = vals == np.inf
        exterior_val = vals.min() * 2 - vals[1 - exterior].max()
        vals[exterior] = exterior_val
        # ground truth
        x = np.arange(0.0, 1+stride, stride)
        y = np.arange(0.0, 1+stride, stride)
        x, y = np.meshgrid(x, y)
        ground_truth = funcs[funcId](x, y)
        ground_truth[exterior] = exterior_val
        # show
        plt.figure(dpi=150)
        plt.subplots_adjust(wspace=0.4)
        ax = plt.subplot(121)
        plt.imshow(vals, origin='lower')
        plt.title('Our Solution')
        plt.colorbar(cax=make_axes_locatable(ax).append_axes("right", size="5%", pad=0.05))
        ax = plt.subplot(122)
        plt.imshow(ground_truth, origin='lower')
        plt.title('Ground Truth')
        plt.colorbar(cax=make_axes_locatable(ax).append_axes("right", size="5%", pad=0.05))
        plt.savefig(outputPath)
        plt.close()

        delta = (vals - ground_truth).flatten()
        norm_1_2_inf = [np.linalg.norm(delta, i) / (n+1)**(2/i) for i in (1, 2, np.inf)]
        # print('1-norm:', norm_1_2_inf[0])
        # print('2-norm:', norm_1_2_inf[1])
        # print('infty-norm:', norm_1_2_inf[2])
        # print('------------------------------------------')
        return norm_1_2_inf

In [4]:
normWriter = open('result/norm.txt', 'w')
for funcId in range(3):
    for regionId in range(2):
        for boundaryType in boundaryTypes[regionId]:
            norms = []
            for n in nRange:
                prefix = f'Func{funcId}_{"HasCircle" if regionId else "NoCircle"}_{boundaryType}_Grid{n}'
                norm_1_2_inf = analyze(funcId, 1/n, f'result/{prefix}.txt', f'fig/{prefix}_img.pdf')
                normWriter.write(f'{funcId}, {"HasCircle" if regionId else "NoCircle"}, {boundaryType}, {n}, {tuple(norm_1_2_inf)}\n')
                norms.append([n] + norm_1_2_inf)
            norms = np.array(norms)
            plt.plot(norms[:, 0], norms[:, 1], label='1-norm')
            plt.plot(norms[:, 0], norms[:, 2], label='2-norm')
            plt.plot(norms[:, 0], norms[:, 3], label='$\\infty$-norm')
            plt.xlabel('Grid Size')
            plt.xticks(nRange)
            plt.legend()
            plt.savefig(f'fig/Func{funcId}_{"HasCircle" if regionId else "NoCircle"}_{boundaryType}_norm.pdf')
            plt.yscale('log')
            plt.xscale('log')
            plt.xticks(nRange)
            plt.savefig(f'fig/Func{funcId}_{"HasCircle" if regionId else "NoCircle"}_{boundaryType}_norm_loglog.pdf')
            plt.close()
normWriter.close()